## Global settings and imports

In [1]:
import sys
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from typing import Dict, List, Optional
import numpy as np
from pathlib import Path
import polars as pl

from utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
)

from utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history,
)
from utils._articles import convert_text2encoding_with_transformers
from utils._polars import concat_str_columns, slice_join_dataframes
from utils._articles import create_article_id_to_value_mapping
from utils._nlp import get_transformers_word_embeddings
from utils._python import write_submission_file, rank_predictions_by_score

from models_pytorch.model_config import hparams_nrms
from models_pytorch.nrms import NRMSModel

from transformers import AutoModel, AutoTokenizer
from models_pytorch.dataloader import NRMSDataLoader


# Check Python version
print(f"Python version: {sys.version.split()[0]}")

# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Current GPU device: {torch.cuda.get_device_name()}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")

/Users/gustavsiphone/Documents/GitHub/Deeplearning-RecSys-Challenge-2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Python version: 3.12.5
PyTorch version: 2.5.1
CUDA available: False


## Prepare parameters

In [2]:
epochs = 5
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'

## Looking at behaviours and history

In [3]:
# Let's first check the data structure
behaviors_path = Path("ebnerd_demo/train/behaviors.parquet")
df = pd.read_parquet(behaviors_path)
print("Behaviors columns:", df.columns)

Behaviors columns: Index(['impression_id', 'article_id', 'impression_time', 'read_time',
       'scroll_percentage', 'device_type', 'article_ids_inview',
       'article_ids_clicked', 'user_id', 'is_sso_user', 'gender', 'postcode',
       'age', 'is_subscriber', 'session_id', 'next_read_time',
       'next_scroll_percentage'],
      dtype='object')


In [4]:
# Debug print for history file structure
history_df = pd.read_parquet("ebnerd_demo/train/history.parquet")
print("\nHistory file columns:", history_df.columns)
print("\nFirst row of history:", history_df.iloc[0])


History file columns: Index(['user_id', 'impression_time_fixed', 'scroll_percentage_fixed',
       'article_id_fixed', 'read_time_fixed'],
      dtype='object')

First row of history: user_id                                                                13538
impression_time_fixed      [2023-04-27T10:17:43.000000, 2023-04-27T10:18:...
scroll_percentage_fixed    [100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...
article_id_fixed           [9738663, 9738569, 9738663, 9738490, 9738663, ...
read_time_fixed            [17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...
Name: 0, dtype: object


## Download and load data

In [5]:
PATH = Path("./ebnerd_demo")  # Base path for your data directory
print(PATH)


ebnerd_demo


In [6]:
def ebnerd_from_path(path: Path, history_size: int = 30) -> pl.DataFrame:
    """
    Load ebnerd - function
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
        .pipe(
            truncate_history,
            column=DEFAULT_HISTORY_ARTICLE_ID_COL,
            history_size=history_size,
            padding_value=0,
            enable_warning=False,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .collect()
        .pipe(
            slice_join_dataframes,
            df2=df_history.collect(),
            on=DEFAULT_USER_COL,
            how="left",
        )
    )
    return df_behaviors

COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
]
HISTORY_SIZE = 10
FRACTION = 0.01

df_train = (
    ebnerd_from_path(PATH.joinpath("train"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(
        sampling_strategy_wu2019,
        npratio=4,
        shuffle=True,
        with_replacement=True,
        seed=123,
    )
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)
# =>
df_validation = (
    ebnerd_from_path(PATH.joinpath("validation"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    .sample(fraction=FRACTION)
)
df_train.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,impression_id,labels
u32,list[i32],list[i64],list[i64],u32,list[i8]
2190812,"[9770533, 9769433, … 9770989]","[9775183, 9775183, … 9774826]",[9772275],467956337,"[0, 0, … 0]"
201433,"[9770207, 9769514, … 9770541]","[9770178, 9769474, … 9771121]",[9771686],489431142,"[0, 0, … 0]"


In [7]:
df_articles = pl.read_parquet(PATH.joinpath("articles.parquet"))
df_articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3037230,"""Ishockey-spiller: Jeg troede j…","""ISHOCKEY: Ishockey-spilleren S…",2023-06-29 06:20:57,false,"""Ambitionerne om at komme til U…",2003-08-28 08:55:00,null,"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Kriminalitet"", ""Kendt"", … ""Mindre ulykke""]",142,"[327, 334]","""sport""",null,null,null,0.9752,"""Negative"""
3044020,"""Prins Harry tvunget til dna-te…","""Hoffet tvang Prins Harry til a…",2023-06-29 06:21:16,false,"""Den britiske tabloidavis The S…",2005-06-29 08:47:00,"[3097307, 3097197, 3104927]","""article_default""","""https://ekstrabladet.dk/underh…","[""Harry"", ""James Hewitt""]","[""PER"", ""PER""]","[""Kriminalitet"", ""Kendt"", … ""Personfarlig kriminalitet""]",414,[432],"""underholdning""",null,null,null,0.7084,"""Negative"""


In [8]:
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
#
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)

# Init dataloaders



In [9]:


train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=64,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)

## Create hyper-parameters

## Train the NRMS model

In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# Define paths
MODEL_NAME = "NRMS"
LOG_DIR = os.path.join("downloads", "runs", MODEL_NAME)
MODEL_WEIGHTS = os.path.join("downloads", "data", "state_dict", MODEL_NAME, "weights.pth")

# Create directories if they don't exist
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(os.path.dirname(MODEL_WEIGHTS), exist_ok=True)

# Define EarlyStopping class
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=2, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
            verbose (bool): If True, prints a message for each validation loss improvement.
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            if self.verbose:
                print(f"Initial validation loss: {self.best_loss:.6f}")
        elif val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.verbose:
                print(f"Validation loss decreased to {self.best_loss:.6f}. Resetting counter.")
        else:
            self.counter += 1
            if self.verbose:
                print(f"No improvement in validation loss for {self.counter} epochs.")
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print("Early stopping triggered.")

# Define ModelCheckpoint class
class ModelCheckpoint:
    """Saves the model after every epoch if it has the best performance so far."""
    def __init__(self, filepath, verbose=False, save_best_only=True):
        """
        Args:
            filepath (str): Path to save the model checkpoint.
            verbose (bool): If True, prints a message when the model is saved.
            save_best_only (bool): If True, saves only when the model is better than before.
        """
        self.filepath = filepath
        self.verbose = verbose
        self.save_best_only = save_best_only
        self.best_loss = None

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss:
            self.best_loss = val_loss
            self.save_checkpoint(model)

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), self.filepath)
        if self.verbose:
            print(f"Model saved to {self.filepath}")

# Initialize TensorBoard SummaryWriter
writer = SummaryWriter(log_dir=LOG_DIR)

# Initialize callbacks
early_stopping = EarlyStopping(patience=2, verbose=True)
model_checkpoint = ModelCheckpoint(filepath=MODEL_WEIGHTS, verbose=True, save_best_only=True)


# Initialize your model
# Ensure that NRMSModel is a PyTorch nn.Module


model = NRMSModel(
    hparams=hparams_nrms.__dict__,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)






NRMSModel(
  (newsencoder): NewsEncoder(
    (embedding): Embedding(250002, 768, padding_idx=0)
    (fc1): Linear(in_features=768, out_features=128, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=128, out_features=200, bias=True)
  )
  (userencoder): UserEncoder(
    (titleencoder): NewsEncoder(
      (embedding): Embedding(250002, 768, padding_idx=0)
      (fc1): Linear(in_features=768, out_features=128, bias=True)
      (relu): ReLU()
      (fc2): Linear(in_features=128, out_features=200, bias=True)
    )
    (self_attention): SelfAttention()
    (attention_layer): AttLayer2(
      (q): Linear(in_features=200, out_features=1, bias=False)
    )
    (user_projection): Linear(in_features=200, out_features=200, bias=True)
  )
)

In [12]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training parameters
NUM_EPOCHS = 1  # Early stopping will handle actual stopping

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
        # Move data to device
        inputs = [inp.to(device) for inp in inputs]
        targets = targets.to(device)

        # Prepare targets
        targets = targets.squeeze()
   #     print(f"Targets after squeeze: {targets}, shape: {targets.shape}")

        # Find positive class index
        positive_indices = (targets == 1).nonzero(as_tuple=False).squeeze()
     #   print(f"Positive indices: {positive_indices}")

        if positive_indices.numel() != 1:
            raise ValueError("Expected exactly one positive label in targets.")

        targets = positive_indices

        # Ensure targets have shape [batch_size]
        if targets.dim() == 0:
            targets = targets.unsqueeze(0)
      #  print(f"Final targets: {targets}, shape: {targets.shape}")

        targets = targets.long()

        optimizer.zero_grad()
        outputs = model(*inputs)
     #   print(f"outputs: {outputs}")
      #  print(f"Outputs shape: {outputs.shape}")  # Should be [batch_size, num_classes]
      #  print(f"Targets shape: {targets.shape}")   # Should be [batch_size]

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if batch_idx % 10 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS}], Step [{batch_idx}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

    avg_train_loss = running_loss / len(train_dataloader)
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)
    print(f"Epoch [{epoch}/{NUM_EPOCHS}], Average Training Loss: {avg_train_loss:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_dataloader:
            inputs = [inp.to(device) for inp in inputs]
            targets = targets.to(device)

            # Process targets as in the training loop
            targets = targets.squeeze()
            print(f"Validation targets after squeeze: {targets}, shape: {targets.shape}")

            positive_indices = (targets == 1).nonzero(as_tuple=False).squeeze()
            print(f"Validation positive indices: {positive_indices}")

            if positive_indices.numel() != 1:
                raise ValueError("Expected exactly one positive label in targets.")

            targets = positive_indices

            if targets.dim() == 0:
                targets = targets.unsqueeze(0)
            print(f"Validation final targets: {targets}, shape: {targets.shape}")

            targets = targets.long()

            outputs = model(*inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    writer.add_scalar('Loss/Validation', avg_val_loss, epoch)
    print(f"Epoch [{epoch}/{NUM_EPOCHS}], Validation Loss: {avg_val_loss:.4f}")

    # Check for improvement
    model_checkpoint(model, avg_val_loss)
    early_stopping(avg_val_loss)

    if early_stopping.early_stop:
        print("Early stopping triggered. Stopping training.")
        break

# Load the best model weights
model.load_state_dict(torch.load(MODEL_WEIGHTS))
print(f"Loaded best model weights from {MODEL_WEIGHTS}")

# Close the TensorBoard writer
writer.close()


Epoch [1/1], Step [0/248], Loss: 1.6151


## Save the model

## Output Prediction File

## References

https://github.com/recommenders-team/recommenders/blob/main/examples/00_quick_start/nrms_MIND.ipynb